## Задание выполнила Колесниченко Ксения Сергеевна.
Контакты https://www.linkedin.com/in/kseniia-kolesnichenko/



# Тестовое задание

В данном задании необходимо написать код, который считает следующие признаки для каждого клиента `user_id`:

1. `events_ordinal_number` - порядковый номер события
2. `second_event_time` - время второго события
2.	`loan_ordinal_number` - порядковый номер займа
2.	`events_time_diff` - разница во времени между событиями
2.	`previous_loans_max_amount` - максимальная сумма предыдущего займа.


### Подготовка датафрейма

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
size = 10000

df = pd.DataFrame(
    {
        'time': [*pd.date_range('2001-01-01', '2020-09-01', freq='1h')][:size],
        'user_id': np.random.choice(1000, size),
        'type': np.random.choice(2, size, p=[0.8, 0.2]),
        'amount': [int(x//1) for x in  np.random.normal(15_000, 1_000, size)],
        'target': np.random.choice(2, size, p=[0.5, 0.5]),
    },
    index=[*range(size)],
)

In [3]:
df['is_loan'] = df['type']
df['type'] = df['type'].replace({0: 'loanRequest', 1: 'loan'})

In [4]:
df.to_csv('test_origin.csv', index=False)

### Расчет признаков

#### Признак `events_ordinal_number`

In [5]:
# весь код расчета признака в этой фукнции
def calculate_events_number(df):
    # выбираем cumcount, потому что он на числах быстрее, чем rank и добавляем 1, чтобы нумерация шла не с 0.
    df['events_ordinal_number'] = df.groupby("user_id").cumcount()+1 
    return df

In [6]:
calculate_events_number(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number
0,2001-01-01 00:00:00,673,loan,14331,0,1,1
1,2001-01-01 01:00:00,968,loanRequest,15905,0,0,1
2,2001-01-01 02:00:00,536,loanRequest,15281,1,0,1
3,2001-01-01 03:00:00,742,loanRequest,11475,1,0,1
4,2001-01-01 04:00:00,20,loanRequest,17310,0,0,1


#### Признак `second_event_time`

In [7]:
# весь код расчета признака в этой фукнции
def calculate_second_event_time(df):
# создаём отдельный датафрейм, где будут взяты только вторые 
    second_event_time= df[df.events_ordinal_number==2] 
# оставляем в нём только нужные столбцы
    second_event_time=second_event_time[['time','user_id']] 
 # переименуем столбец
    second_event_time.rename(columns={"time": "second_event_time"}, inplace=True)
# Присоединим к основному датафрейму время второго вхождения
    df=df.merge(second_event_time, left_on='user_id', right_on='user_id')    
    return df

In [8]:
calculate_second_event_time(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,second_event_time
0,2001-01-01 00:00:00,673,loan,14331,0,1,1,2001-03-07 14:00:00
1,2001-03-07 14:00:00,673,loanRequest,15161,1,0,2,2001-03-07 14:00:00
2,2001-03-31 00:00:00,673,loanRequest,14195,1,0,3,2001-03-07 14:00:00
3,2001-05-17 20:00:00,673,loanRequest,14228,1,0,4,2001-03-07 14:00:00
4,2001-06-28 02:00:00,673,loanRequest,15406,0,0,5,2001-03-07 14:00:00


#### Признак `loan_ordinal_number`

In [9]:
# весь код расчета признака должен быть в этом методе
def calculate_loan_number(df):
    
#сортируем данные строго по user_id и времени, переиндексируем весь датафрейм
    df = df.sort_values(['user_id', 'time'], ignore_index=True)
#нумеруем выданные кредиты в зависимости от времени выдачи (только событие type = loan)
#метод cumcount() работает проще на не числовых значениях, чем rank()
#+1 ставим, потому что иначе начнет нумеровать с нуля
    df['loan_ordinal_number'] = df[df['type']=='loan'].groupby(['user_id'])['time'].cumcount()+1
#по результатам предыдущего шага в датафрейме остается куча NaN'ов для loanRequest'ов
#заполняем эти NaN'ы следующими значениями _назад_ (метод bfill)
    df['loan_ordinal_number'] = df.groupby('user_id')['loan_ordinal_number'].fillna(method='bfill')
#у нас остаются NaN'ы в loanRequest'ах, которые не закончились выдачей кредита. Присваиваем им значение +1 от последнего кредита
#методом ffill (вперед), но кидаем его во временный столбец, чтобы +1 не убил уже присвоенные значения RowNum
    df['RowNum2'] = df.groupby('user_id')['loan_ordinal_number'].fillna(method='ffill')+1
#еще раз проходим по нашему столбцу RowNum и вытягиваем в него значения из временного RowNum2 только там, где остался NaN
    df['loan_ordinal_number'] = df['loan_ordinal_number'].fillna(df['RowNum2'])
#убиваем временный столбец
    df = df.drop('RowNum2',1)

    return df

In [10]:
calculate_loan_number(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,loan_ordinal_number
0,2001-03-14 15:00:00,0,loanRequest,14713,0,0,1,1.0
1,2001-04-30 22:00:00,0,loanRequest,16486,1,0,2,1.0
2,2001-06-06 12:00:00,0,loan,14057,0,1,3,1.0
3,2001-06-17 12:00:00,0,loanRequest,14438,1,0,4,2.0
4,2001-07-13 08:00:00,0,loan,14553,1,1,5,2.0


In [11]:
def calculate_loan_number(df):
    df = df.sort_values(['user_id', 'time'], ignore_index=True)
    df['loan_ordinal_number'] = df[df['type']=='loan'].groupby(['user_id'])['time'].cumcount()+1
    df['loan_ordinal_number'] = df.groupby('user_id')['loan_ordinal_number'].fillna(method='bfill')
    df['RowNum2'] = df.groupby('user_id')['loan_ordinal_number'].fillna(method='ffill')+1
    df['loan_ordinal_number'] = df['loan_ordinal_number'].fillna(df['RowNum2'])
    df = df.drop({'RowNum2'}, axis=1)

    return df

In [12]:
calculate_loan_number(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,loan_ordinal_number
0,2001-03-14 15:00:00,0,loanRequest,14713,0,0,1,1.0
1,2001-04-30 22:00:00,0,loanRequest,16486,1,0,2,1.0
2,2001-06-06 12:00:00,0,loan,14057,0,1,3,1.0
3,2001-06-17 12:00:00,0,loanRequest,14438,1,0,4,2.0
4,2001-07-13 08:00:00,0,loan,14553,1,1,5,2.0


#### Признак `events_time_diff`

In [13]:
# весь код расчета признака должен быть в этом методе
def calculate_time_diff(df):
# Приводим формат времени к datetime
    df['time']=pd.to_datetime(df['time'], format="%Y-%m-%d %H:%M:%S") 
# Укажем, как считать events_time_diff
    df['events_time_diff']=df.groupby(['user_id'])['time'].diff()

    return df

In [14]:
calculate_time_diff(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,events_time_diff
0,2001-01-01 00:00:00,673,loan,14331,0,1,1,NaT
1,2001-01-01 01:00:00,968,loanRequest,15905,0,0,1,NaT
2,2001-01-01 02:00:00,536,loanRequest,15281,1,0,1,NaT
3,2001-01-01 03:00:00,742,loanRequest,11475,1,0,1,NaT
4,2001-01-01 04:00:00,20,loanRequest,17310,0,0,1,NaT


#### Признак `previous_loans_max_amount`

In [15]:
# весь код расчета признака должен быть в этом методе
def calculate_previous_max_amount(df):

    df['previous_loans_max_amount'] =  df.groupby('user_id')['amount'].expanding(1).max().reset_index(level=0, drop=True).shift(1)

    return df

In [16]:
calculate_previous_max_amount(df).head()

,time,user_id,type,amount,target,is_loan,events_ordinal_number,events_time_diff,previous_loans_max_amount
0,2001-01-01 00:00:00,673,loan,14331,0,1,1,NaT,17237.0
1,2001-01-01 01:00:00,968,loanRequest,15905,0,0,1,NaT,16413.0
2,2001-01-01 02:00:00,536,loanRequest,15281,1,0,1,NaT,16341.0
3,2001-01-01 03:00:00,742,loanRequest,11475,1,0,1,NaT,16447.0
4,2001-01-01 04:00:00,20,loanRequest,17310,0,0,1,NaT,16661.0


##### При применении функций новый столбец появляется. Но он не всегда присоединяется к датафрейму. Для более надёжного результата лучше использовать 

df=calculate_events_number(df)

df=calculate_second_event_time(df)

df=calculate_loan_number(df)

df=calculate_time_diff(df)

df=calculate_previous_max_amount(df)

### Сохранение итогового датасета с признаками

Этот код трогать не нужно.

In [17]:
def calculate_feature(df):
    calculate_events_number(df)
    calculate_second_event_time(df)
    calculate_loan_number(df)
    calculate_time_diff(df)
    calculate_previous_max_amount(df)
    

In [ ]:
df

In [19]:
%%timeit -o 
calculate_feature(df)

419 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<TimeitResult : 419 ms ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)>

In [20]:
columns = ['time']
rez = _
df_time = pd.DataFrame([rez], None, columns)
df_time.to_csv('time.csv', index=False)

In [21]:
df.to_csv('test.csv', index=False)